In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from tqdm.keras import TqdmCallback

## Red ball Prediction

In [11]:
# 红球超参
epochs      = 1000  # 训练轮数
batch_size  = 10    # 批大小
layer       = 1000  # 隐藏层大小
patience    = 20    # EarlyStopping 的耐心值

# 优化器
adam_optimizer = Adam(learning_rate=0.0001)

In [12]:
import openpyxl
# 读取数据
data = pd.read_excel('双色球.xlsx')
input_features = data[['Ball1', 'Ball2', 'Ball3', 'Ball4', 'Ball5', 'Ball6']]

# One-Hot编码
label_binarizer = LabelBinarizer()
label_binarizer.fit(range(1, 34))
input_features_one_hot = label_binarizer.transform(input_features.values.flatten()).reshape(-1, 6, 33).sum(axis=1)

In [13]:
# 构建红球模型
model = Sequential([
    Input(shape=(33,)),                                         # 输入层，输入维度为 33
    Dense(128, activation='relu'),                              # 第一隐藏层，128 个神经元，ReLU 激活函数
    *[Dense(64, activation='relu') for _ in range(layer)],      # 中间隐藏层，每层 64 个神经元，ReLU 激活函数
    Dense(33, activation='sigmoid')                             # 输出层，33 个神经元，Sigmoid 激活函数
])

# 编译红球模型
model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# 红球模型训练
model.fit(input_features_one_hot,   # 输入特征
          input_features_one_hot,   # 输出标签（自编码器的目标是重建输入）
          epochs=epochs,            # 训练轮数
          batch_size=batch_size,    # 批大小
          verbose=0,                # 不显示训练过程
          callbacks=[TqdmCallback(verbose=1),  # 显示进度条
                     EarlyStopping(monitor='loss', patience=patience)])  # EarlyStopping 回调

In [ ]:
# 红球预测
predictions = model.predict(input_features_one_hot[-1].reshape(1, -1))
predicted_indices = np.argsort(predictions[0])[::-1][:6]  # 选择最高的6个概率
predicted_balls = np.array(range(1, 34))[predicted_indices]

## Blue Ball Prediction

In [ ]:
# 读取数据
data = pd.read_excel('双色球.xlsx')
blue_balls = data['BlueBall'].values.reshape(-1, 1)

# 归一化
scaler = MinMaxScaler(feature_range=(0, 1))
blue_balls_scaled = scaler.fit_transform(blue_balls)

In [ ]:
# 构建蓝球模型
model = Sequential()
model.add(Input(shape=(1,)))
model.add(Dense(128, activation='relu'))
for _ in range(layer):
    model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # 输出层，1个输出

# 编译建蓝球模型
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# 蓝球模型训练
model.fit(blue_balls_scaled, 
            blue_balls_scaled, 
            epochs=epochs, 
            batch_size=batch_size,
            verbose=0,
            callbacks=[TqdmCallback(verbose=1), EarlyStopping(monitor='loss', patience=patience)]
            )

In [ ]:
# 蓝球预测
prediction = model.predict(blue_balls_scaled[-1].reshape(1, -1))
predicted_ball = scaler.inverse_transform(prediction).astype(int)

## 预测结果

In [ ]:
# 输出预测结果
print('Predicted Red Balls:', predicted_balls)
print('Predicted Blue Ball:', predicted_ball.flatten()[0])